In [37]:
from ipywidgets import widgets
from IPython.display import display
from datetime import datetime
import maped
import pandas as pd
import datetime


In [20]:
import ipywidgets as widgets
from IPython.display import display
import datetime

# --- Create the DatePicker widget ---
date_picker = widgets.DatePicker(
    description='Select Date:',
    disabled=False,
    value=datetime.date(2025, 2, 27) # Example initial date
)

# --- Display the DatePicker ---
display(date_picker)

DatePicker(value=datetime.date(2025, 2, 27), description='Select Date:')

In [38]:


date_picker.value.strftime("%Y-%m-%d")

'2025-02-27'

In [43]:
observatory_code_input = widgets.Dropdown(
    options=pd.read_csv('C:/Users/michi/OneDrive/Escritorio/phase_curves_project/MPC_observatory_codes/MPC_OBS_CODES.csv').Code.to_list(),
    value='Y28',
    description='Observatory code:',
    style={'description_width': 'initial'}
)
display(observatory_code_input)



Dropdown(description='Observatory code:', index=2500, options=('000', '001', '002', '003', '004', '005', '006'…

Dropdown(description='Observatory code:', index=2500, options=('000', '001', '002', '003', '004', '005', '006'…

In [35]:
import logi
MPC_CODES_FILE = 'C:/Users/michi/OneDrive/Escritorio/phase_curves_project/MPC_observatory_codes/MPC_OBS_CODES.csv'
site_longitude_360, site_latitude, site_elevation_m = logi.get_observatory_coordinates(MPC_CODES_FILE, observatory_code_input.value)
# Convert 0-360 E to -180 to +180 E
site_longitude = site_longitude_360
if site_longitude > 180.0:
    site_longitude = site_longitude - 360.0 

print(f"Observatory {observatory_code_input.value}: {site_latitude:.4f}°N, {site_longitude:.4f}°E, {site_elevation_m}m")

### utc offset from coordinates
utc_offset, tz_name = logi.get_utc_offset_from_coords(site_latitude, site_longitude)
print(f"UTC Offset: {utc_offset:+.1f} hours")

Observatory I33: -30.2379°N, -70.7337°E, 2776.132m
UTC Offset: -3.0 hours


In [5]:
widgets.Text(value='Hello World!', disabled=True)

Text(value='Hello World!', disabled=True)

In [39]:
# 2. Create the Text Input Widget
coordinates_input = widgets.Text(
    value='14 09 46.7 +01 59 47', # Initial value
    placeholder= "HH MM SS.SS ±DD MM SS.S",
    description='Coordinates:',
    style={'description_width': 'initial'}
)

display(coordinates_input)

Text(value='14 09 46.7 +01 59 47', description='Coordinates:', placeholder='HH MM SS.SS ±DD MM SS.S', style=De…

In [40]:
coordinates_input.value

'14 09 46.7 +01 59 47'

In [8]:
widgets.Dropdown(
    options=[('One', 1), ('Two', 2), ('Three', 3)],
    value=2,
    description='Number:',
)

Dropdown(description='Number:', index=1, options=(('One', 1), ('Two', 2), ('Three', 3)), value=2)

In [11]:
widgets.Dropdown(
    options=['1', '2', '3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31'],
    value='1',
    description='day:',
    disabled=False,
)

Dropdown(description='day:', options=('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14…

In [12]:
widgets.Dropdown(
    options=[('January', 1), ('February', 2), ('March', 3),
             ('April', 4), ('May', 5), ('June', 6),
             ('July', 7), ('August', 8), ('September', 9),
             ('October', 10), ('November', 11), ('December', 12)],
    value=2,
    description='month:',
)

Dropdown(description='month:', index=1, options=(('January', 1), ('February', 2), ('March', 3), ('April', 4), …

In [13]:
widgets.Dropdown(
    options=[str(1985+i) for i in range(2031-1985)],
    value='2025',
    description='year:',
    disabled=False,
)    

Dropdown(description='year:', index=40, options=('1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992…

In [14]:
import ipywidgets as widgets
from IPython.display import display

# --- 1. Day Dropdown ---
day_options = [str(i) for i in range(1, 32)]
day_dropdown = widgets.Dropdown(
    options=day_options,
    value='1',
    description='Day:',
    disabled=False,
)

# --- 2. Month Dropdown ---
month_options = [
    ('January', 1), ('February', 2), ('March', 3),
    ('April', 4), ('May', 5), ('June', 6),
    ('July', 7), ('August', 8), ('September', 9),
    ('October', 10), ('November', 11), ('December', 12)
]
month_dropdown = widgets.Dropdown(
    options=month_options,
    value=2,
    description='Month:',
)

# --- 3. Year Dropdown ---
year_options = [str(i) for i in range(1985, 2031)]
year_dropdown = widgets.Dropdown(
    options=year_options,
    value='2025',
    description='Year:',
    disabled=False,
)

# --- 4. Combine with HBox ---
date_selector = widgets.HBox([day_dropdown, month_dropdown, year_dropdown])

# --- 5. Display the combined widget ---
display(date_selector)

In [18]:
import ipywidgets as widgets
from IPython.display import display
import datetime

# --- Create the DatePicker widget ---
date_picker = widgets.DatePicker(
    description='Select Date:',
    disabled=False,
    value=datetime.date(2025, 2, 27) # Example initial date
)

# --- Display the DatePicker ---
display(date_picker)

DatePicker(value=datetime.date(2025, 2, 27), description='Select Date:')

In [19]:
date_picker.value

datetime.date(2025, 2, 27)

In [8]:

# --------------------------------------------------------------------------
# --- Widget Implementation ---
# --------------------------------------------------------------------------

# 1. Define the main function that handles widget input
def calculate_jd_with_widget(date_string: str):
    """
    Processes the date string from the widget, calls the parser,
    calculates Julian Date, and prints the result.
    """
    # Clear previous output
    output_area.clear_output(wait=True)
    
    with output_area:
        print(f"Input String: {date_string}")
        print("-" * 30)
        
        try:
            # Call your flexible parser
            Y, M, D_decimal = maped.parse_date_to_YMD_flexible(date_string)
            
            # Print parsed components
            print(f"Parsed Y, M, D(decimal): ({Y}, {M}, {D_decimal:.8f})")
            
            # Call the Julian Date function
            julian_date = maped.calculate_julian_day(Y, M, D_decimal)
            
            # Print final result
            print("-" * 30)
            print(f"✨ Julian Date (JD): {julian_date:.8f}")
            
        except ValueError as e:
            # Handle parsing errors and display them
            print(f"❌ ERROR: {e}")
        except Exception as e:
            # Handle any other unexpected errors
            print(f"🛑 UNEXPECTED ERROR: {e}")


# 2. Create the Text Input Widget
date_time_input = widgets.Text(
    value='2025-10-06 13:30:45.123', # Initial value
    placeholder='YYYY-MM-DD HH:MM:SS.SS',
    description='Enter UTC Time:',
    style={'description_width': 'initial'}
)

# 3. Create an output area to display results
output_area = widgets.Output()

# 4. Create an interactive object linked to the input widget
# We use the interactive function to automatically link the widget's value 
# to the function's argument.
interactive_widget = widgets.interactive(
    calculate_jd_with_widget, 
    date_string=date_time_input
)

# 5. Display the widget and the output area
display(interactive_widget, output_area)

interactive(children=(Text(value='2025-10-06 13:30:45.123', description='Enter UTC Time:', placeholder='YYYY-M…

Output()

In [1]:
from geopy.geocoders import Nominatim
from datetime import datetime
import pytz

def get_utc_offset(lat, lon):
    geolocator = Nominatim(user_agent="timezone_app")
    location = geolocator.reverse(f"{lat}, {lon}", language='en')
    
    # This gives you the location, but not directly the timezone
    # You'd still need a timezone lookup
    print(location.raw)

# This approach has limitations without timezone data

In [1]:
from tzfpy import get_tz
import pytz
from datetime import datetime

def get_utc_offset_from_coords(lat, lon):
    """
    Get UTC offset from latitude and longitude using tzfpy
    
    Args:
        lat: Latitude (float)
        lon: Longitude (float)
        
    Returns:
        tuple: (offset_hours, timezone_name) or (None, None) if failed
    """
    try:
        # Get timezone name from coordinates
        tz_name = get_tz(lon, lat)  # Note: tzfpy uses (lon, lat) order
        
        if tz_name is None:
            print("Could not find timezone for these coordinates")
            return None, None
        
        # Get the timezone object
        tz = pytz.timezone(tz_name)
        
        # Get current UTC offset (including DST if applicable)
        now = datetime.now(tz)
        offset = now.utcoffset().total_seconds() / 3600
        
        return offset, tz_name
        
    except Exception as e:
        print(f"Error: {e}")
        return None, None


# Example usage
if __name__ == "__main__":
    # Test with different coordinates
    test_locations = [
        (40.7128, -74.0060, "New York"),
        (51.5074, -0.1278, "London"),
        (-23.5505, -46.6333, "São Paulo"),
        (35.6762, 139.6503, "Tokyo"),
        (-22.9068, -43.1729, "Rio de Janeiro"),
    ]
    
    for lat, lon, name in test_locations:
        print(f"\n--- {name} ---")
        print(f"Coordinates: ({lat}, {lon})")
        
        offset, tz_name = get_utc_offset_from_coords(lat, lon)
        
        if offset is not None:
            print(f"UTC Offset: {offset:+.1f} hours")
            print(f"Timezone: {tz_name}")
            
            # Show current time in that timezone
            tz = pytz.timezone(tz_name)
            local_time = datetime.now(tz)
            print(f"Current time: {local_time.strftime('%Y-%m-%d %H:%M:%S %Z')}")
        else:
            print("Could not determine UTC offset")



--- New York ---
Coordinates: (40.7128, -74.006)
UTC Offset: -4.0 hours
Timezone: America/New_York
Current time: 2025-10-06 13:33:58 EDT

--- London ---
Coordinates: (51.5074, -0.1278)
UTC Offset: +1.0 hours
Timezone: Europe/London
Current time: 2025-10-06 18:33:58 BST

--- São Paulo ---
Coordinates: (-23.5505, -46.6333)
UTC Offset: -3.0 hours
Timezone: America/Sao_Paulo
Current time: 2025-10-06 14:33:58 -03

--- Tokyo ---
Coordinates: (35.6762, 139.6503)
UTC Offset: +9.0 hours
Timezone: Asia/Tokyo
Current time: 2025-10-07 02:33:58 JST

--- Rio de Janeiro ---
Coordinates: (-22.9068, -43.1729)
UTC Offset: -3.0 hours
Timezone: America/Sao_Paulo
Current time: 2025-10-06 14:33:58 -03


In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# # Create a sample DataFrame
# df = pd.read_csv('C:/Users/michi/OneDrive/Escritorio/phase_curves_project/MPC_observatory_codes/MPC_OBS_CODES.csv')

# Create a sample DataFrame
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David'],
    'Age': [25, 30, 35, 40],
    'City': ['New York', 'London', 'Paris', 'Tokyo']
}
df = pd.DataFrame(data)

# Define a function to display the selected row
def select_row_by_name(name):
    selected_row = df[df['Name'] == name]
    display(selected_row)

# Create a Dropdown widget with options from the 'Name' column
name_dropdown = widgets.Dropdown(
    options=df['Name'].tolist(),
    value=df['Name'].iloc[0],  # Set initial value
    description='Select Name:'
)

# Link the dropdown to the function using interactive
widgets.interactive(select_row_by_name, name=name_dropdown)

# Display the dropdown widget
display(name_dropdown)

Dropdown(description='Select Name:', options=('Alice', 'Bob', 'Charlie', 'David'), value='Alice')